In [ ]:
from google.colab import drive
import os
import time
import pandas as pd

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -U bitsandbytes
!pip install Levenshtein sqlparse

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 63.8 MB/s eta 0:00:00


In [ ]:
# === Imports ===
import os
import json
import pandas as pd
from tqdm import tqdm

# For Hugging Face models
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# For OpenAI/Groq API
from openai import OpenAI

# Preprocessing

## Dataset Import

In [ ]:
!pip install datasets pandas tqdm tabulate

In [ ]:
from datasets import load_dataset
import pandas as pd
from tqdm import tqdm
import json


### WikiSql Import

In [ ]:
# === Load WikiSQL-like dataset from Hugging Face ===
wikisql = load_dataset("b-mc2/sql-create-context", split="train")

print(f"✅ Loaded WikiSQL-like dataset with {len(wikisql)} samples.")
print("Columns:", wikisql.column_names)

# === Preview a sample ===
sample = wikisql[0]
print("\n🔹 Example WikiSQL sample:")
print(json.dumps(sample, indent=2))


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

sql_create_context_v4.json:   0%|          | 0.00/21.8M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/78577 [00:00<?, ? examples/s]

✅ Loaded WikiSQL-like dataset with 78577 samples.
Columns: ['answer', 'question', 'context']

🔹 Example WikiSQL sample:
{
  "answer": "SELECT COUNT(*) FROM head WHERE age > 56",
  "question": "How many heads of the departments are older than 56 ?",
  "context": "CREATE TABLE head (age INTEGER)"
}


### Spider Import

In [ ]:
# === Load Spider test set from Hugging Face ===
spider = load_dataset("spider", split="validation")

print(f"✅ Loaded Spider test split with {len(spider)} samples.")
print("Columns:", spider.column_names)

# === Preview a sample ===
sample = spider[0]
print("\n🔹 Example Spider sample:")
print(json.dumps(sample, indent=2))


README.md: 0.00B [00:00, ?B/s]

spider/train-00000-of-00001.parquet:   0%|          | 0.00/831k [00:00<?, ?B/s]

spider/validation-00000-of-00001.parquet:   0%|          | 0.00/126k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1034 [00:00<?, ? examples/s]

✅ Loaded Spider test split with 1034 samples.
Columns: ['db_id', 'query', 'question', 'query_toks', 'query_toks_no_value', 'question_toks']

🔹 Example Spider sample:
{
  "db_id": "concert_singer",
  "query": "SELECT count(*) FROM singer",
  "question": "How many singers do we have?",
  "query_toks": [
    "SELECT",
    "count",
    "(",
    "*",
    ")",
    "FROM",
    "singer"
  ],
  "query_toks_no_value": [
    "select",
    "count",
    "(",
    "*",
    ")",
    "from",
    "singer"
  ],
  "question_toks": [
    "How",
    "many",
    "singers",
    "do",
    "we",
    "have",
    "?"
  ]
}


### Smaller Dataset for faster testing

In [ ]:
sample_wikisql = wikisql.select(range(100))
sample_spider = spider.select(range(100))

print("✅ Subsets prepared: 100 samples each.")


✅ Subsets prepared: 100 samples each.


In [ ]:
sample_wikisql.to_json("datasets/wikisql_test_100.json")
sample_spider.to_json("datasets/spider_test_100.json")

print("✅ Saved local dataset subsets.")


Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

✅ Saved local dataset subsets.


In [ ]:
print(f"WikiSQL samples: {len(sample_wikisql)}")
print(f"Spider samples: {len(sample_spider)}")

print("\nColumns available:")
print("WikiSQL:", wikisql.column_names)
print("Spider:", spider.column_names)


WikiSQL samples: 100
Spider samples: 100

Columns available:
WikiSQL: ['answer', 'question', 'context']
Spider: ['db_id', 'query', 'question', 'query_toks', 'query_toks_no_value', 'question_toks']


## Model Download

In [ ]:
import os
import subprocess

# HF models to clone (skip llama3-sqlcoder-8b if already on Drive)
hf_models = {
    "sqlcoder-7b-2": "https://huggingface.co/defog/sqlcoder-7b-2",
    # "llama3-3b-instruct": "https://huggingface.co/meta-llama/Llama-3.2-3B-Instruct"
}

drive_models_folder = "/content/drive/MyDrive/Major_Project/models"
os.makedirs(drive_models_folder, exist_ok=True)

for name, repo in hf_models.items():
    drive_path = os.path.join(drive_models_folder, name)

    if os.path.exists(drive_path):
        print(f"{name} already exists at {drive_path}, skipping clone.")
        continue

    print(f"\nCloning {name} directly to Drive...")
    subprocess.run(["git", "clone", repo, drive_path])
    print(f"✅ Saved {name} to Drive at {drive_path}")

print("\n✅ All required HF models are now on Drive.")



Cloning sqlcoder-7b-2 directly to Drive...
✅ Saved sqlcoder-7b-2 to Drive at /content/drive/MyDrive/Major Project/models/sqlcoder-7b-2

Cloning llama3-3b-instruct directly to Drive...
✅ Saved llama3-3b-instruct to Drive at /content/drive/MyDrive/Major Project/models/llama3-3b-instruct

✅ All required HF models are now on Drive.


## Normalization

In [ ]:
import json

# === Normalize WikiSQL samples ===
def normalize_wikisql_sample(sample):
    """
    Convert b-mc2/sql-create-context format to standard benchmark format.
    Fields:
    - question: NL question
    - sql: target SQL string
    - table_schema: list of column headers
    """
    normalized = {
        "question": sample["question"],
        "sql": sample.get("answer", sample.get("sql")),  # field name may differ
        "table_schema": sample.get("context", []),      # column headers
        "db_id": None  # WikiSQL has single table only
    }
    return normalized

wikisql_normalized = [normalize_wikisql_sample(s) for s in wikisql]
print(f"✅ Normalized {len(wikisql_normalized)} WikiSQL samples.")
print(json.dumps(wikisql_normalized[0], indent=2))


✅ Normalized 78577 WikiSQL samples.
{
  "question": "How many heads of the departments are older than 56 ?",
  "sql": "SELECT COUNT(*) FROM head WHERE age > 56",
  "table_schema": "CREATE TABLE head (age INTEGER)",
  "db_id": null
}


In [ ]:
# === Normalize Spider samples ===
def normalize_spider_sample(sample):
    """
    Convert Spider validation split sample to standard benchmark format.
    Fields:
    - question: NL question
    - sql: gold SQL
    - table_schema: table column headers (flattened)
    - db_id: database ID for execution evaluation
    """
    # Flatten all table headers
    headers = []
    if "db_id" in sample:
        # Spider schema not directly in dataset, you can add later using DB introspection
        # For now just placeholder
        headers = sample.get("columns", []) if "columns" in sample else []

    normalized = {
        "question": sample["question"],
        "sql": sample["query"],
        "table_schema": headers,
        "db_id": sample["db_id"]
    }
    return normalized

spider_normalized = [normalize_spider_sample(s) for s in spider]
print(f"✅ Normalized {len(spider_normalized)} Spider samples.")
print(json.dumps(spider_normalized[0], indent=2))


✅ Normalized 1034 Spider samples.
{
  "question": "How many singers do we have?",
  "sql": "SELECT count(*) FROM singer",
  "table_schema": [],
  "db_id": "concert_singer"
}


In [ ]:
all_data = wikisql_normalized + spider_normalized
print(f"✅ Total samples for benchmarking: {len(all_data)}")


✅ Total samples for benchmarking: 79611


In [ ]:
import os
import json

# === Create a results folder if it doesn't exist ===
os.makedirs("datasets_normalized", exist_ok=True)

# === Save normalized WikiSQL ===
with open("datasets_normalized/wikisql_normalized.json", "w") as f:
    json.dump(wikisql_normalized, f, indent=2)
print("✅ Saved WikiSQL normalized dataset to datasets_normalized/wikisql_normalized.json")

# === Save normalized Spider ===
with open("datasets_normalized/spider_normalized.json", "w") as f:
    json.dump(spider_normalized, f, indent=2)
print("✅ Saved Spider normalized dataset to datasets_normalized/spider_normalized.json")

# === Optional: Save merged dataset ===
with open("datasets_normalized/all_data_normalized.json", "w") as f:
    json.dump(all_data, f, indent=2)
print("✅ Saved merged normalized dataset to datasets_normalized/all_data_normalized.json")


✅ Saved WikiSQL normalized dataset to datasets_normalized/wikisql_normalized.json
✅ Saved Spider normalized dataset to datasets_normalized/spider_normalized.json
✅ Saved merged normalized dataset to datasets_normalized/all_data_normalized.json


In [ ]:
import os
import json

# === Base project folder in Drive ===
project_folder = "/content/drive/MyDrive/Major_Project"
datasets_folder = os.path.join(project_folder, "datasets")
os.makedirs(datasets_folder, exist_ok=True)

# === Save sample subsets ===
sample_wikisql.to_json(os.path.join(datasets_folder, "wikisql_test_100.json"), orient="records", lines=True)
sample_spider.to_json(os.path.join(datasets_folder, "spider_test_100.json"), orient="records", lines=True)
print("✅ Saved sample dataset subsets to Drive:", datasets_folder)

# === Folder for normalized datasets ===
normalized_folder = os.path.join(project_folder, "datasets_normalized")
os.makedirs(normalized_folder, exist_ok=True)

# === Save normalized WikiSQL ===
with open(os.path.join(normalized_folder, "wikisql_normalized.json"), "w") as f:
    json.dump(wikisql_normalized, f, indent=2)
print("✅ Saved WikiSQL normalized dataset to Drive:", normalized_folder)

# === Save normalized Spider ===
with open(os.path.join(normalized_folder, "spider_normalized.json"), "w") as f:
    json.dump(spider_normalized, f, indent=2)
print("✅ Saved Spider normalized dataset to Drive:", normalized_folder)

# === Optional: Save merged dataset ===
with open(os.path.join(normalized_folder, "all_data_normalized.json"), "w") as f:
    json.dump(all_data, f, indent=2)
print("✅ Saved merged normalized dataset to Drive:", normalized_folder)


Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

✅ Saved sample dataset subsets to Drive: /content/drive/MyDrive/Major Project/datasets
✅ Saved WikiSQL normalized dataset to Drive: /content/drive/MyDrive/Major Project/datasets_normalized
✅ Saved Spider normalized dataset to Drive: /content/drive/MyDrive/Major Project/datasets_normalized
✅ Saved merged normalized dataset to Drive: /content/drive/MyDrive/Major Project/datasets_normalized


# Unified Model Interface & Load Model

In [ ]:
from getpass import getpass
import os

# Prompt for Groq API key
groq_key = getpass("Enter your Groq API Key: ")
os.environ["GROQ_API_KEY"] = groq_key

print("✅ Groq API key set.")

# Prompt for Mistral API key
# mistral_key = getpass("Enter your Mistral API Key: ")
# os.environ["MISTRAL_API_KEY"] = mistral_key

# print("✅ Mistral API key set.")



Enter your Groq API Key: ··········
✅ Groq API key set.


In [ ]:
# === Load Normalized Datasets from Drive ===
project_folder = "/content/drive/MyDrive/Major_Project"
normalized_folder = os.path.join(project_folder, "datasets_normalized")

wikisql_normalized_path = os.path.join(normalized_folder, "wikisql_normalized.json")
spider_normalized_path = os.path.join(normalized_folder, "spider_normalized.json")
all_data_normalized_path = os.path.join(normalized_folder, "all_data_normalized.json")

with open(wikisql_normalized_path, "r") as f:
    wikisql_normalized = json.load(f)

with open(spider_normalized_path, "r") as f:
    spider_normalized = json.load(f)

with open(all_data_normalized_path, "r") as f:
    all_data = json.load(f)

print("✅ Loaded normalized datasets from Drive.")
print(f"WikiSQL samples: {len(wikisql_normalized)}")
print(f"Spider samples: {len(spider_normalized)}")
print(f"Total samples: {len(all_data)}")

✅ Loaded normalized datasets from Drive.
WikiSQL samples: 78577
Spider samples: 1034
Total samples: 79611


In [ ]:
class BaseModel:
    """
    Unified interface for all text-to-SQL models.
    Each model must implement `generate_sql(question, table_schema)`.
    """
    def __init__(self, name):
        self.name = name

    def generate_sql(self, question, table_schema):
        raise NotImplementedError


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

class HFModel(BaseModel):
    def __init__(self, model_path, use_gpu=True, load_in_4bit=True):
        """
        model_path : str
            HF model name or path (e.g., "defog/sqlcoder-7b-2" or local folder)
        use_gpu : bool
            Whether to load model on GPU
        load_in_4bit : bool
            Whether to use 4-bit quantization (recommended for large models)
        """
        super().__init__(model_path)

        # ✅ FIXED: use model_path, not model
        short_name = model_path.split("/")[-1]
        self.model_path = f"/content/drive/MyDrive/Major_Project/models/{short_name}"

        self.device_map = "auto" if use_gpu else None
        self.use_gpu = use_gpu
        self.load_in_4bit = load_in_4bit

        print(f"Loading HF model from {self.model_path} on {'GPU' if use_gpu else 'CPU'}...")
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_path)

        quantization_config = None
        if load_in_4bit:
            quantization_config = BitsAndBytesConfig(
                load_in_4bit=True,
                bnb_4bit_use_double_quant=True,
                bnb_4bit_quant_type="nf4",
                bnb_4bit_compute_dtype=torch.float16
            )

        self.model = AutoModelForCausalLM.from_pretrained(
            self.model_path,
            device_map=self.device_map,
            quantization_config=quantization_config
        )
        print("✅ Model loaded successfully.")

    def generate_sql(self, question, table_schema, max_new_tokens=256):
        """
        Generates SQL query for a given question and table schema.
        """
        prompt = (
            f"You are an expert SQL generator. Give Query only. No formatting also.\n"
            f"Columns: {table_schema}\nQuestion: {question}\nSQL:"
        )

        inputs = self.tokenizer(prompt, return_tensors="pt").to(self.model.device)

        with torch.no_grad():
            outputs = self.model.generate(
                **inputs,
                max_new_tokens=max_new_tokens,
                do_sample=False
            )

        # ✅ Only decode new tokens (skip prompt)
        gen_tokens = outputs[0][inputs["input_ids"].shape[1]:]
        pred_sql = self.tokenizer.decode(gen_tokens, skip_special_tokens=True)
        return pred_sql.strip()


In [ ]:
class GroqModel(BaseModel):
    def __init__(self, model_name):
        super().__init__(model_name)
        from openai import OpenAI
        self.client = OpenAI(api_key=os.getenv("GROQ_API_KEY"), base_url="https://api.groq.com/openai/v1")

    def generate_sql(self, question, table_schema):
        prompt = f"You are an expert SQL generator. Give Query only. No formating also. \nColumns: {table_schema}\nQuestion: {question}\nSQL:"
        resp = self.client.chat.completions.create(
            model=self.name,
            messages=[{"role": "user", "content": prompt}],
            temperature=0
        )
        # Print the response structure for debugging
        # print("Groq API Response:", resp)
        return resp.choices[0].message.content.strip()

In [ ]:
class MistralModel(BaseModel):
    def __init__(self, model_name):
        super().__init__(model_name)
        from openai import OpenAI
        self.client = OpenAI(
            api_key=os.getenv("MISTRAL_API_KEY"),
            base_url="https://api.mistral.ai/v1"
        )

    def generate_sql(self, question, table_schema):
        prompt = (
            f"You are an expert SQL generator. Give Query only. No formatting also.\n"
            f"Columns: {table_schema}\nQuestion: {question}\nSQL:"
        )

        resp = self.client.chat.completions.create(
            model=self.name,
            messages=[{"role": "user", "content": prompt}],
            temperature=0
        )

        # --- Robust extraction of text ---
        choice = resp.choices[0]

        if hasattr(choice, "message"):
            # message can be object or list
            msg = choice.message
            if isinstance(msg, list):
                pred_sql = msg[0].content
            else:
                pred_sql = msg.content
        else:
            # fallback for older response structure
            pred_sql = choice.text

        return pred_sql.strip()


# Evaluation Loop

In [ ]:
import sqlparse

def normalize_sql(sql):
    """Normalize SQL string for comparison."""
    return sqlparse.format(sql, keyword_case="upper", strip_comments=True).strip()

def exact_match(pred, gold):
    return normalize_sql(pred) == normalize_sql(gold)


In [ ]:
# Folder for benchmark results
drive_results_path = "/content/drive/MyDrive/Major_Project/results"
os.makedirs(drive_results_path, exist_ok=True)
print(f"✅ Results folder ready: {drive_results_path}")


✅ Results folder ready: /content/drive/MyDrive/Major_Project/results


In [ ]:
import time
import pandas as pd
import Levenshtein # You might need to install this library: pip install Levenshtein

def run_benchmark_with_metrics(model, dataset, max_samples=None):
    """
    Runs a model on a dataset, returns per-sample results with latency and string match percentage.
    """
    results = []
    total_string_match_percentage = 0

    for i, sample in enumerate(dataset[:max_samples] if max_samples else dataset):
        question = sample["question"]
        table_schema = sample.get("columns", sample.get("table_schema", []))
        gold_sql = sample.get("sql") or sample.get("query") or ""

        start_time = time.time()
        pred_sql = model.generate_sql(question, table_schema)
        latency = time.time() - start_time

        # Calculate string match percentage using Levenshtein distance
        # Normalize both strings before calculating similarity
        normalized_pred_sql = pred_sql.strip().lower()
        normalized_gold_sql = gold_sql.strip().lower()

        # Avoid division by zero if gold_sql is empty
        if len(normalized_gold_sql) == 0:
            string_match_percentage = 100.0 if len(normalized_pred_sql) == 0 else 0.0
        else:
            distance = Levenshtein.distance(normalized_pred_sql, normalized_gold_sql)
            max_len = max(len(normalized_pred_sql), len(normalized_gold_sql))
            string_match_percentage = (1 - (distance / max_len)) * 100 if max_len > 0 else 100.0


        total_string_match_percentage += string_match_percentage

        results.append({
            "question": question,
            "gold_sql": gold_sql,
            "pred_sql": pred_sql,
            "latency_sec": latency,
            "string_match_percentage": string_match_percentage
        })

        if (i + 1) % 10 == 0:
            print(f"Processed {i+1}/{max_samples} samples")

    df = pd.DataFrame(results)

    # Calculate average string match percentage
    if max_samples and max_samples > 0:
        avg_string_match_percentage = total_string_match_percentage / max_samples
    elif len(dataset) > 0:
        avg_string_match_percentage = total_string_match_percentage / len(dataset)
    else:
        avg_string_match_percentage = 0

    print(f"Average String Match Percentage: {avg_string_match_percentage:.2f}%")

    return df

In [ ]:
# === Run models and save metrics ===
max_samples = 50
# === Define models by provider ===
mistral_models = ["magistral-small-2509", "mistral-small-2506"]
groq_models = ["llama-3.1-8b-instant", "llama-3.3-70b-versatile", "openai/gpt-oss-120b", "openai/gpt-oss-20b"]
hf_models = ["defog/llama-3-sqlcoder-8b", "defog/sqlcoder-7b-2"]

# === Datasets ===
datasets_dict = {
    "spider": spider_normalized,
    "wikisql": wikisql_normalized,
    # "all_data": all_data  # merged normalized dataset
}

for model_type, models in [
    # ("mistral", mistral_models),
    # ("groq", groq_models),
    ("hf", hf_models)
    ]:
    for model_name in models:
        print(f"\n=== Running {model_type.upper()} model: {model_name} ===")
        model_class = {"mistral": MistralModel, "groq": GroqModel, "hf": HFModel}[model_type]
        model = model_class(model_name)

        for dataset_name, dataset in datasets_dict.items():
            print(f"--- Dataset: {dataset_name} ({len(dataset)} samples) ({max_samples} tested) ---")
            df = run_benchmark_with_metrics(model, dataset, max_samples=max_samples)

            filename = f"{drive_results_path}/{model_name.replace('/', '_')}_{dataset_name}_{model_type}_results.csv"
            df.to_csv(filename, index=False)
            print(f"✅ Saved results to {filename}")



=== Running HF model: defog/llama-3-sqlcoder-8b ===
Loading HF model from /content/drive/MyDrive/Major_Project/models/llama-3-sqlcoder-8b on GPU...


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


✅ Model loaded successfully.
--- Dataset: spider (1034 samples) (50 tested) ---


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

Processed 30/50 samples


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Processed 40/50 samples


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Processed 50/50 samples
Average String Match Percentage: 65.38%
✅ Saved results to /content/drive/MyDrive/Major_Project/results/defog_sqlcoder-7b-2_wikisql_hf_results.csv


In [ ]:
# === Generate and Save Summary Table from Results Files ===
summary_records = []
drive_results_path = "/content/drive/MyDrive/Major_Project/results"

# Iterate through all files in the results directory
for filename in os.listdir(drive_results_path):
    if filename.endswith("_results.csv"):
        filepath = os.path.join(drive_results_path, filename)

        try:
            df = pd.read_csv(filepath)

            # Extract model name and dataset name from the filename based on the saving structure
            # Filename structure: {model_name.replace('/', '_')}_{dataset_name}_{model_class}_results.csv
            parts = filename.replace('_results.csv', '').split('_')

            # The parts should be [model_name_part1, model_name_part2, ..., dataset_name, model_class_name]
            # We need to find where the dataset name starts. It's usually the second to last part
            # before the model class name.
            if len(parts) < 3:
                 print(f"Skipping file with unexpected format: {filename}")
                 continue

            # Assume the dataset name is the second to last element, and model class is the last
            dataset_name = parts[-2]
            # Reconstruct model name, handling potential underscores within the original model name
            model_name_parts = parts[:-2]
            model_name = "/".join(model_name_parts).replace('_', '/') if len(model_name_parts) > 1 else model_name_parts[0].replace('_', '/')


            # Calculate average metrics
            if not df.empty:
                avg_latency_sec = df["latency_sec"].mean()
                # Check if the column exists before accessing
                if "string_match_percentage" in df.columns:
                    average_string_match_percentage = df["string_match_percentage"].mean()
                else:
                    average_string_match_percentage = None # Or some other indicator

                # Determine model_type (groq or hf) from the model_name
                model_type = parts[-1]


                summary_records.append({
                    "model_type": model_type,
                    "model_name": model_name,
                    "dataset": dataset_name,
                    "avg_latency_sec": avg_latency_sec,
                    "average_string_match_percentage": average_string_match_percentage
                })
            else:
                print(f"Warning: Results file {filename} is empty.")

        except Exception as e:
            print(f"Error processing file {filename}: {e}")


if summary_records:
    summary_df = pd.DataFrame(summary_records)
    summary_csv = f"{drive_results_path}/summary_metrics.csv"
    summary_df.to_csv(summary_csv, index=False)
    print(f"\n✅ Generated and saved summary metrics to {summary_csv}")
    display(summary_df)
else:
    print("\nNo valid results files found to generate summary.")


✅ Generated and saved summary metrics to /content/drive/MyDrive/Major_Project/results/summary_metrics.csv


,model_type,model_name,dataset,avg_latency_sec,average_string_match_percentage
0,groq,llama-3.1-8b-instant,spider,1.067156,69.029412
1,groq,llama-3.1-8b-instant,wikisql,2.627981,82.961204
2,groq,llama-3.3-70b-versatile,spider,1.003520,69.908427
3,groq,llama-3.3-70b-versatile,wikisql,2.253361,84.242272
4,groq,openai/gpt-oss-120b,spider,2.005409,60.085232
5,groq,openai/gpt-oss-120b,wikisql,2.945870,75.513050
6,groq,openai/gpt-oss-20b,spider,1.437410,57.494380
7,groq,openai/gpt-oss-20b,wikisql,2.459313,73.714249
8,hf,defog/llama-3-sqlcoder-8b,spider,2.808553,52.280616
9,hf,defog/llama-3-sqlcoder-8b,wikisql,2.644874,71.788852
